In [1]:
import warnings

In [2]:
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from keras.models import load_model

In [4]:
import pandas as pd

In [5]:
# create an object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [6]:
# importing the GL Bot corpus file for pre-processing

words=[]
classes = []
documents = []
ignore_words = ['?', '!']

In [9]:
data_file = open("data.json").read()
intents = json.loads(data_file)

In [10]:
intents

{'intents': [{'tag': 'Welcome',
   'patterns': ['just going to say hi',
    'heya',
    'hello hi',
    'howdy',
    'hey there',
    'hi there',
    'greetings',
    'hey',
    'long time no see',
    'hello',
    "lovely day isn't it",
    'I greet you',
    'hello again',
    'hi',
    'hello there',
    'A good day',
    'hy',
    'hello',
    'hola',
    'how are you'],
   'responses': ['Hi! How are you doing?',
    'Hello! How can I help you?',
    'Good day! What can I do for you today?',
    'Greetings! How can I assist?',
    'Hello and welcome!',
    'Hello, and thank you for reaching out!']},
  {'tag': 'CourseOverview',
   'patterns': ['Tell me about the Computer Science program.',
    'Please Tell me about the course overview.',
    "What's the general course overview?",
    'Can you provide a summary of the course content?',
    'please provide me course overview.',
    'Tell me about the course overview',
    'Tell me about the course',
    'course overview',
    'Please 